# Samba Insight - Exploratory Data Analysis

**Project:** Brazilian E-Commerce Analytics

**Purpose:** Exploratory analysis of BigQuery warehouse data to uncover insights and patterns.

**Data Sources:**
- `fact_orders` - Order-level transactions
- `mart_sales_daily` - Pre-aggregated daily sales
- `mart_customer_cohorts` - Customer cohort analysis

---

In [ ]:
# Import libraries
import sys
import warnings
from pathlib import Path

# Add project root to path for importing local modules
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from google.cloud import bigquery
from google.oauth2 import service_account

warnings.filterwarnings("ignore")

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

print("✅ Libraries imported successfully")

In [ ]:
# Initialize BigQuery client
import os

# Try to import config from project utils
try:
    from src.utils.config import get_config

    config = get_config()
    project_id = config.gcp_project_id
    credentials_path = config.google_application_credentials
except ImportError:
    # Fallback to environment variables
    credentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
    project_id = os.getenv("GCP_PROJECT_ID", "project-samba-insight")

# Get dataset name from environment or use default
dataset = os.getenv("BQ_DATASET_WAREHOUSE", "dev_warehouse_warehouse")

if not credentials_path or not os.path.exists(credentials_path):
    print("⚠️  Warning: Using application default credentials")
    client = bigquery.Client(project=project_id)
else:
    credentials = service_account.Credentials.from_service_account_file(
        credentials_path,
        scopes=["https://www.googleapis.com/auth/bigquery"],
    )
    client = bigquery.Client(project=project_id, credentials=credentials)

print(f"✅ Connected to BigQuery project: {project_id}")
print(f"📊 Using dataset: {dataset}")

## 1. Data Overview

Load key metrics and understand the data landscape.

In [ ]:
# Query: Overall business metrics
query_overview = f"""
SELECT
  COUNT(DISTINCT order_id) as total_orders,
  COUNT(DISTINCT customer_key) as total_customers,
  ROUND(SUM(total_order_value), 2) as total_revenue,
  ROUND(AVG(total_order_value), 2) as avg_order_value,
  ROUND(AVG(review_score), 2) as avg_review_score,
  MIN(order_purchase_date) as first_order_date,
  MAX(order_purchase_date) as last_order_date,
  DATE_DIFF(MAX(order_purchase_date), MIN(order_purchase_date), DAY) as days_of_data
FROM `{project_id}.{dataset}.fact_orders`
WHERE order_status = 'delivered'
"""

df_overview = client.query(query_overview).to_dataframe()
print("\n📊 Business Overview:")
print("=" * 60)
for col in df_overview.columns:
    print(f"{col:.<40} {df_overview[col].iloc[0]}")
print("=" * 60)

## 2. Revenue Analysis

Analyze revenue trends and patterns over time.

In [ ]:
# Query: Monthly revenue trends
query_revenue = f"""
SELECT
  FORMAT_DATE('%Y-%m', month_date) as month,
  SUM(total_revenue) as revenue,
  SUM(total_orders) as orders,
  AVG(avg_order_value) as aov
FROM `{project_id}.{dataset}.mart_sales_monthly`
GROUP BY month_date, month
ORDER BY month_date
"""

df_revenue = client.query(query_revenue).to_dataframe()

# Plot revenue trend
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Revenue trend
ax1.plot(df_revenue["month"], df_revenue["revenue"], marker="o", linewidth=2, color="#1f77b4")
ax1.set_title("Monthly Revenue Trend", fontsize=16, fontweight="bold")
ax1.set_xlabel("Month", fontsize=12)
ax1.set_ylabel("Revenue (R$)", fontsize=12)
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis="x", rotation=45)

# Order volume trend
ax2.bar(df_revenue["month"], df_revenue["orders"], color="#2ca02c", alpha=0.7)
ax2.set_title("Monthly Order Volume", fontsize=16, fontweight="bold")
ax2.set_xlabel("Month", fontsize=12)
ax2.set_ylabel("Orders", fontsize=12)
ax2.grid(True, alpha=0.3, axis="y")
ax2.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

print(
    f"\n💡 Revenue grew from R${df_revenue['revenue'].iloc[0]:,.0f} to R${df_revenue['revenue'].iloc[-1]:,.0f}"
)
print(
    f"💡 Total growth: {((df_revenue['revenue'].iloc[-1] / df_revenue['revenue'].iloc[0]) - 1) * 100:.1f}%"
)

## 3. Customer Segmentation Analysis

Understand customer segments and their behaviors.

In [ ]:
# Query: Customer segments
query_segments = f"""
SELECT
  customer_segment,
  COUNT(*) as customers,
  AVG(total_orders) as avg_orders,
  AVG(total_orders * 150) as estimated_ltv,
  AVG(avg_review_score) as avg_rating
FROM `{project_id}.{dataset}.dim_customer`
WHERE total_orders > 0
GROUP BY customer_segment
ORDER BY customers DESC
"""

df_segments = client.query(query_segments).to_dataframe()

# Plot customer segments
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Segment distribution
colors = ["#ff7f0e", "#2ca02c", "#d62728"]
ax1.pie(
    df_segments["customers"],
    labels=df_segments["customer_segment"],
    autopct="%1.1f%%",
    colors=colors,
    startangle=90,
)
ax1.set_title("Customer Segment Distribution", fontsize=14, fontweight="bold")

# LTV by segment
ax2.bar(df_segments["customer_segment"], df_segments["estimated_ltv"], color=colors, alpha=0.7)
ax2.set_title("Estimated Lifetime Value by Segment", fontsize=14, fontweight="bold")
ax2.set_ylabel("LTV (R$)", fontsize=12)
ax2.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

print("\n📊 Customer Segment Summary:")
print(df_segments.to_string(index=False))

## 4. Product Category Performance

Identify top-performing product categories.

In [ ]:
# Query: Top product categories
query_categories = f"""
SELECT
  product_category,
  SUM(total_orders) as orders,
  SUM(total_revenue) as revenue,
  AVG(avg_review_score) as avg_rating
FROM `{project_id}.{dataset}.mart_product_performance`
WHERE product_category IS NOT NULL
GROUP BY product_category
ORDER BY revenue DESC
LIMIT 15
"""

df_categories = client.query(query_categories).to_dataframe()

# Plot top categories
fig, ax = plt.subplots(figsize=(12, 8))

y_pos = np.arange(len(df_categories))
ax.barh(y_pos, df_categories["revenue"], color="#1f77b4", alpha=0.7)
ax.set_yticks(y_pos)
ax.set_yticklabels(df_categories["product_category"])
ax.invert_yaxis()
ax.set_xlabel("Revenue (R$)", fontsize=12)
ax.set_title("Top 15 Product Categories by Revenue", fontsize=16, fontweight="bold")
ax.grid(True, alpha=0.3, axis="x")

plt.tight_layout()
plt.show()

print(
    f"\n🏆 Top Category: {df_categories['product_category'].iloc[0]} - R${df_categories['revenue'].iloc[0]:,.0f}"
)

## 5. Geographic Analysis

Analyze sales distribution across Brazilian states.

In [ ]:
# Query: Sales by state
query_geo = f"""
SELECT
  customer_state,
  SUM(total_orders) as orders,
  SUM(total_revenue) as revenue,
  COUNT(DISTINCT customer_city) as cities
FROM `{project_id}.{dataset}.mart_sales_daily`
GROUP BY customer_state
ORDER BY revenue DESC
LIMIT 10
"""

df_geo = client.query(query_geo).to_dataframe()

# Plot geographic distribution
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(df_geo))
width = 0.35

ax.bar(x - width / 2, df_geo["orders"], width, label="Orders", color="#2ca02c", alpha=0.7)
ax2 = ax.twinx()
ax2.plot(x, df_geo["revenue"], marker="o", color="#d62728", linewidth=2, label="Revenue")

ax.set_xlabel("State", fontsize=12)
ax.set_ylabel("Orders", fontsize=12, color="#2ca02c")
ax2.set_ylabel("Revenue (R$)", fontsize=12, color="#d62728")
ax.set_title("Top 10 States by Orders and Revenue", fontsize=16, fontweight="bold")
ax.set_xticks(x)
ax.set_xticklabels(df_geo["customer_state"])
ax.legend(loc="upper left")
ax2.legend(loc="upper right")
ax.grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

print("\n🗺️  Top 5 States:")
print(df_geo[["customer_state", "orders", "revenue"]].head().to_string(index=False))

## 6. Cohort Retention Analysis

Analyze customer retention patterns by cohort.

In [ ]:
# Query: Cohort retention
query_retention = f"""
SELECT
  cohort_month_name,
  months_since_cohort,
  retention_rate_pct,
  cohort_size
FROM `{project_id}.{dataset}.mart_customer_retention`
WHERE cohort_month >= '2017-01-01'
  AND months_since_cohort <= 12
ORDER BY cohort_month_name, months_since_cohort
"""

# Pivot for heatmap
df_retention = client.query(query_retention).to_dataframe()

df_retention_agg = df_retention.groupby(
    ["cohort_month_name", "months_since_cohort"], as_index=False
).agg({"retention_rate_pct": "mean"})

pivot_retention = df_retention_agg.pivot(
    index="cohort_month_name", columns="months_since_cohort", values="retention_rate_pct"
)

# Plot retention heatmap
fig, ax = plt.subplots(figsize=(14, 8))

sns.heatmap(
    pivot_retention,
    annot=True,
    fmt=".1f",
    cmap="RdYlGn",
    vmin=0,
    vmax=100,
    cbar_kws={"label": "Retention Rate (%)"},
    ax=ax,
)
ax.set_title("Customer Retention Heatmap by Cohort", fontsize=16, fontweight="bold")
ax.set_xlabel("Months Since First Order", fontsize=12)
ax.set_ylabel("Cohort Month", fontsize=12)

plt.tight_layout()
plt.show()

print("\n💡 Retention Insights:")
print(f"  • Month 1 Retention: {pivot_retention[1].mean():.1f}% (average)")
print(f"  • Month 6 Retention: {pivot_retention[6].mean():.1f}% (average)")
print(f"  • Month 12 Retention: {pivot_retention[12].mean():.1f}% (average)")

## 7. Key Insights Summary

Automated insights generation based on the analysis.

In [ ]:
print("\n" + "=" * 80)
print("🎯 KEY INSIGHTS - SAMBA INSIGHT ANALYSIS")
print("=" * 80)

print("\n📊 Business Performance:")
print(f"  • Total Revenue: R${df_overview['total_revenue'].iloc[0]:,.0f}")
print(f"  • Total Orders: {df_overview['total_orders'].iloc[0]:,}")
print(f"  • Average Order Value: R${df_overview['avg_order_value'].iloc[0]:.2f}")
print(f"  • Customer Satisfaction: {df_overview['avg_review_score'].iloc[0]:.2f}/5.0 ⭐")

print("\n👥 Customer Insights:")
print(f"  • Total Customers: {df_overview['total_customers'].iloc[0]:,}")
loyal_pct = (
    df_segments[df_segments["customer_segment"] == "loyal"]["customers"].iloc[0]
    / df_segments["customers"].sum()
    * 100
)
print(f"  • Loyal Customers: {loyal_pct:.1f}% of customer base")
print(
    f"  • Highest LTV Segment: {df_segments.loc[df_segments['estimated_ltv'].idxmax(), 'customer_segment']}"
)

print("\n🏆 Product Insights:")
print(f"  • Top Category: {df_categories['product_category'].iloc[0]}")
print(f"  • Top Category Revenue: R${df_categories['revenue'].iloc[0]:,.0f}")
print(
    f"  • Category Concentration: Top 3 categories = {(df_categories['revenue'].head(3).sum() / df_categories['revenue'].sum() * 100):.1f}% of revenue"
)

print("\n🗺️  Geographic Insights:")
print(f"  • Top State: {df_geo['customer_state'].iloc[0]}")
print(f"  • Top State Revenue: R${df_geo['revenue'].iloc[0]:,.0f}")
print(
    f"  • Geographic Concentration: Top 3 states = {(df_geo['revenue'].head(3).sum() / df_geo['revenue'].sum() * 100):.1f}% of revenue"
)

print("\n🔄 Retention Insights:")
if "pivot_retention" in locals():
    print(f"  • 1-Month Retention: {pivot_retention[1].mean():.1f}%")
    print(f"  • 6-Month Retention: {pivot_retention[6].mean():.1f}%")
    print(f"  • 12-Month Retention: {pivot_retention[12].mean():.1f}%")

print("\n" + "=" * 80)
print("\n✅ Analysis Complete")
print(
    f"📅 Data Period: {df_overview['first_order_date'].iloc[0]} to {df_overview['last_order_date'].iloc[0]}"
)
print(f"📈 Total Days: {df_overview['days_of_data'].iloc[0]}")

## 8. Export Key Metrics

Export insights for reporting and communication.

In [ ]:
# Create summary dataframe for export
summary_metrics = {
    "Metric": [
        "Total Revenue",
        "Total Orders",
        "Avg Order Value",
        "Total Customers",
        "Avg Review Score",
        "Top Product Category",
        "Top State",
    ],
    "Value": [
        f"R${df_overview['total_revenue'].iloc[0]:,.0f}",
        f"{df_overview['total_orders'].iloc[0]:,}",
        f"R${df_overview['avg_order_value'].iloc[0]:.2f}",
        f"{df_overview['total_customers'].iloc[0]:,}",
        f"{df_overview['avg_review_score'].iloc[0]:.2f}/5.0",
        df_categories["product_category"].iloc[0],
        df_geo["customer_state"].iloc[0],
    ],
}

df_summary = pd.DataFrame(summary_metrics)

# Ensure reports directory exists
reports_dir = project_root / "reports"
reports_dir.mkdir(parents=True, exist_ok=True)

# Export to CSV
output_path = reports_dir / "eda_summary_metrics.csv"
df_summary.to_csv(output_path, index=False)

print(f"\n✅ Summary metrics exported to: {output_path}")
print("\n📊 Summary Metrics:")
print(df_summary.to_string(index=False))